In [2]:
import requests
import re
from tqdm import tqdm
import time
import string
import random

In [78]:
basepath = 'https://quote-citation.com/author/confucius/page/{}'
html_re = '''<p>(.+?)</p>\n'''
paths = [basepath.format(i) for i in range(1,6)]
'''<div class="quote-text"><div class="quote-text-inner" id="quote-text-inner-1055">'''

'<div class="quote-text"><div class="quote-text-inner" id="quote-text-inner-1055">'

In [82]:
confuc = []
for path in tqdm(paths):
    try:
        r = requests.get(path)
        if r.status_code == 200:
            chunk = re.findall(html_re, r.text, re.DOTALL)
            confuc += chunk
        else: 
            print(path, r.status_code)
    except Exception as e:
        print(path, e)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:03<00:00,  1.54it/s]


In [99]:
len(confuc)

94

In [ ]:
%%time
for i, p in enumerate(confuc):
    replacements = [
        ('\s?Ср\.\s?', ' '),
        ('\(.*?\)', ''),
        ('(\S) +(\.)', '\g<1>\g<2>'),
        ('[a-zA-Z\\t\\n\\r]*', ''),
        ('^\W+(\w+)', '\g<1>'),
        (' +', ' '),
    ]
    for old, new in replacements:
        p = re.sub(old, new, p)
    confuc[i] = p

In [95]:
path2 = 'https://ru.wikiquote.org/wiki/%D0%9A%D0%BE%D0%BD%D1%84%D1%83%D1%86%D0%B8%D0%B9'
rt = requests.get(path2).text
html_re2 = '''<li>([А-Яа-я0-9\,\.\!\?\s\:\;\—\&\#]{3,})\s?</li>'''
confuc2 = re.findall(html_re2, rt, re.DOTALL)

In [96]:
%%time
for i, p in enumerate(confuc2):
    replacements = [
        ('\s?Ср\.\s?', ' '),
        ('\(.*?\)', ''),
        ('(\S) +(\.)', '\g<1>\g<2>'),
        ('&#160;',' '),
        ('[a-zA-Z\\t\\n\\r\&\#0-9]*', ''),
        ('^\W+(\w+)', '\g<1>'),
        (' +', ' '),
    ]
    for old, new in replacements:
        p = re.sub(old, new, p)
    confuc2[i] = p

Wall time: 9.05 ms


In [97]:
len(confuc2)

74

In [98]:
confuc_all = list(set(confuc + confuc2))
len(confuc_all)

109

In [100]:
with open('confuc.txt', 'w', encoding='utf-8') as f:
    for i in confuc_all:
        f.write("{}\n".format(i))